In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import logging

import numpy as np
import pandas as pd
from hydra import compose, initialize
# from omegaconf import OmegaConf
import matplotlib.pyplot as plt

from physped.io.readers import trajectory_reader
from physped.preprocessing.trajectory_preprocessor import preprocess_trajectories
from physped.core.trajectory_simulator import simulate_trajectories
from physped.visualization.plot_trajectories import plot_trajectories
from physped.core.functions_to_discretize_grid import learn_potential_from_trajectories
from physped.io.readers import read_grid_bins
from physped.io.writers import save_piecewise_potential
from physped.visualization.histograms import create_all_histograms, plot_multiple_histograms

log = logging.getLogger(__name__)

In [ ]:
# Initialize Hydra and read configuration
initialize(version_base=None, config_path="conf", job_name="test_app")


In [ ]:
cfg = compose(config_name="params/custom_paths", return_hydra_config=True, overrides=["params.folder_path=${hydra.runtime.cwd}/data/models/notebook/"])

In [ ]:
# !wget https://zenodo.org/record/7007358/files/glow19_pnasnexus.csv
trajectories = pd.read_csv('glow19_pnasnexus.csv')
trajectories['time'] = pd.to_datetime(trajectories['time'])
# trajectories = trajectories[['time', 'tracked_object', 'x', 'y']].copy()
trajectories.head(3)

# Update parameters
cfg.params.dt = 0.0333333333333
cfg.params.colnames.xf = 'x_sav'
cfg.params.colnames.yf = 'y_sav'
cfg.params.colnames.Pid = 'tracked_object'
cfg.params.colnames.time = 'time'

# Infer the edge of the measurement domain from the data
xmin = int(np.floor(trajectories.x.min())) - 1
xmax = int(np.ceil(trajectories.x.max())) + 1
ymin = int(np.floor(trajectories.y.min())) - 1
ymax = int(np.ceil(trajectories.y.max())) + 1
cfg.params.trajectory_plot.xlims = [xmin, xmax]
cfg.params.trajectory_plot.ylims = [ymin, ymax]

In [ ]:
# Preprocess trajectories
preprocessed_trajectories = preprocess_trajectories(trajectories, parameters=cfg.params)
preprocessed_trajectories.head(3)

In [ ]:
plot_trajectories(preprocessed_trajectories, cfg.params, 'recorded')

In [ ]:
def create_custom_paths_grid(xbins, ybins, rbins, thetabins, kbins = np.array([0, 1, 10**10])):
    np.savez(file=f"data/grids/custom_paths_grid.npz", x=xbins, y=ybins, r=rbins, theta=thetabins, k=kbins)
    
create_custom_paths_grid(
    xbins = np.arange(xmin, xmax, 0.5),
    ybins = np.arange(ymin, ymax, 0.5),
    rbins = np.arange(0, 2, 0.4),
    thetabins = np.linspace(-np.pi, np.pi + 0.01, 6),
    kbins = np.array([0,1,10**10])
)

In [ ]:
# Learn piecewise potential from the trajectories
grid_bins = read_grid_bins(cfg.params.grid_name)
piecewise_potential = learn_potential_from_trajectories(preprocessed_trajectories, grid_bins)
save_piecewise_potential(piecewise_potential, Path(cfg.params.folder_path))


In [ ]:
# Simulate new trajectories using the learned potential
simulated_trajectories = simulate_trajectories(piecewise_potential, cfg.params)
simulated_trajectories.head(3)

In [ ]:
plot_trajectories(simulated_trajectories, cfg.params, 'simulated')  

In [ ]:

cfg.params.histogram_plot.xlims = [xmin, xmax]
cfg.params.histogram_plot.ylims = [ymin, ymax]
# Create histograms
observables = ["xf", "yf", "rf", "thetaf"]
histograms = create_all_histograms(preprocessed_trajectories, simulated_trajectories, observables = ["xf", "yf", "rf", "thetaf"])

# Plot Histograms
plot_multiple_histograms(observables, histograms, "PDF", cfg.params)

In [ ]:
# import itertools package
import itertools
from itertools import permutations 
 
# initialize lists
list_1 = ["a", "b", "c","d"]
list_2 = [1,4,9]
 
# create empty list to store the
# combinations
unique_combinations = []
 
# Getting all permutations of list_1 
# with length of list_2
permut = itertools.permutations(list_1, len(list_2))
 
# zip() is called to pair each permutation
# and shorter list element into combination
for comb in permut:
    zipped = zip(comb, list_2)
    unique_combinations.append(list(zipped))
 
# printing unique_combination list 
print(unique_combinations)